In [ ]:
#Import libraries
from ipy_table import *
from IPython.display import HTML, display, clear_output, Markdown, Javascript
import IPython.core.display as di
import IPython.display as display
import subprocess
import urllib.request, json, urllib.error
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import sys
from utils import *
import datetime
import time
import pandas as pd
import IPython
import IPython.core.display as di
from IPython.display import display
from IPython.display import Markdown as md
from IPython.display import Image 
import os
import requests
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from pandas.io.json import json_normalize
import tabulate
import sys
from nbparameterise import extract_parameters, parameter_values, replace_definitions
import nbformat

#set some display style
display(HTML("<style>.container { width:100% !important; }</style>"))
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)
pd.options.display.max_rows = 4000

if not os.path.exists('BTK_Outputs/img/'):
    os.makedirs('BTK_Outputs/img/')
else:
    pass

if not os.path.exists('img'):
    os.makedirs('img')
else:
    pass

#Check whether page exists & catch errors
with open("BTKNB_FINAL.ipynb") as f:
    nb = nbformat.read(f, as_version=4)

orig_parameters = extract_parameters(nb)

with urllib.request.urlopen("https://blobtoolkit.genomehubs.org/api/v1/search/all") as url:
    data = json.loads(url.read().decode())

for element in data:
    id = element['id']
    #Return GCA accession of sample in question
    with urllib.request.urlopen("https://blobtoolkit.genomehubs.org/api/v1/dataset/id/" + str(id) + "/assembly") as accUrl:
        accData = json.loads(accUrl.read().decode())
        
    gcaString = accData["accession"]
    print("Running for assembly", id)
    
    with urllib.request.urlopen("https://blobtoolkit.genomehubs.org/api/v1/summary/" + id) as sumUrl:
        data = json.loads(sumUrl.read().decode())
        df = json_normalize(data['summaryStats'],sep="_")
    with urllib.request.urlopen("https://blobtoolkit.genomehubs.org/api/v1/dataset/id/" + id + "/reads") as readUrl:
        readData = json.loads(readUrl.read().decode())
        readDf = json_normalize(readData)
    if df.empty or readDf.empty:
        pass
    else:
    
        #Update the parameters and run the notebook
        params = parameter_values(orig_parameters, id=id)
        new_nb = replace_definitions(nb, params)

        # Save
        with open("BTK_Outputs/%s.ipynb" % gcaString, 'w') as f:
            nbformat.write(new_nb, f)

       # subprocess.call('jupyter nbconvert --to html --ExecutePreprocessor.timeout=-1 --execute BTK_Outputs/BTKNB_FINAL_' + id +'.ipynb')
